## ***Libraries Used***

In [49]:
# --------------------------
import numpy as np
import os
import sys
from tqdm import tqdm
import pandas as pd
import logging
import math
# --------------------------
import matplotlib.pyplot as plt
import random
from collections import deque
import tensorflow as tf
import keras.backend as K
from keras.models import Sequential
from keras.models import load_model, clone_model
from keras.layers import Dense
from keras.optimizers import Adam

#---------------------------
from docopt import docopt
import coloredlogs

## ***Huber Loss Function***
   
$$L_\delta(f(x), y) = $$


In [17]:
def huber_loss(true_vale, estimate, delta):
    error = estimate - true_value
    cond = K.abs(error) <= delta
    square_error = 0.5*K.square(error)
    quadratic_error = delta*(K.abs(error) - 0.5*delta)
    return K.mean(np.where(cond, square_error, quadratic_error))
    

## ***Abstracted Agent Class***

In [88]:
class Agent:
    def __init__(self, state_size, strategy = 't-dqn', reset_every = 1000, pretrained = False, model_name = None):
        
        # agent configuration 
        self.strategy = strategy
        self.state_size = state_size
        self.action_size = 3 # [hold, buy, sell]
        self.inventory = []
        self.memory = deque(maxlen = 10000)
        self.first_iter = True
        
        # model configuration
        self.model_name = model_name
        self.gamma = 0.95 # how much importance you want to give for future rewards
        self.epsilon = 1.0 # epsilon greedy strategy parameter
        self.epsilon_min = 0.1
        self.epsilon_decay_rate = 0.995
        self.learning_rate = 0.001
        self.loss = huber_loss # loss function --> Huber Loss function better that mean square error because it is less sensitive to outliers
        self.custom_objects = {"huber_loss": huber_loss}
        self.optimizers = Adam(learning_rate = self.learning_rate)
        
        if pretrained and self.model_name is not None:
            self.model = self.load()
        else:
            self.model = self._model()
            
        # strategy configuration
        if self.strategy in ["t-dqn", "double_dqn"]:
            self.n_iter = 1
            self.reset_every = reset_every
            
            # target network
            self.target_model = clone_model(self.model)
            self.target_model.set_weights(self.model.get_weights())
            
    def _model(self):
        
        model = Sequential([])
        model.add(Dense(units=128, activation = 'relu', input_dim = self.state_size))
        model.add(Dense(units=256, activation = 'relu'))
        model.add(Dense(units=256, activation = 'relu'))
        model.add(Dense(units=128, activation = 'relu'))
        model.add(Dense(units=self.action_size))
                  
        model.compile(loss=self.loss, optimizer = self.optimizers)
        print(model.summary())
        return model
        
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((self, state, action, reward, next_state, done))
                  
    def act(self, state, is_eval = False):
        print(state)
        if not is_eval and random.random() <= self.epsilon:
            return random.randrange(self.action_size)

        if self.first_iter:
            self.first_iter = False
            return 1 # make a definite buy on first iteration
                  
        action_probs = self.model.predict(np.array(state))
        return np.argmax(actions_probs[0]) # actions corresponding to the largest Q value

    def train_experience_replay(self, batch_size):
        mini_batch = random.sample(self.memory, batch_size)
        X_train, y_train = [], []
                  
        if(self.strategy == "dqn"):
            for state, action, reward, next_state, done in mini_batch:
                if(done):
                  target = reward
                else:
                  target = reward + self.gamma*np.amax(self.model.predict(next_state)[0]) # [0] why??

                q_values = self.model.predict(state)
                q_values[0][action] = target
                X_train.append(state[0])
                y_train.append(q_values[0])
                  
                  
        elif(self.strategy == 't-dqn'):
            if(self.n_iter%self.reset_every == 0):
                  self.target_model.set_weights(self.model.get_weights())
                  
            for state, actions, reward, next_state, done in mini_batch:
                if(done):
                    target = reward
                else:
                    target = reward + self.gamma*np.amax(self.target_model.predict(next_state)[0])
                  
            q_values = self.model.predict(state)
            q_values[0][action] = target
                  
            X_train.append(state[0])
            y_train.append(q_values[0])
                  
        elif(self.strategy == 'double-dqn'):
            if(self.n_iter%self.reset_every == 0):
              self.target_model.setweights(self.model.getweights())
            
            for state, action, reward, next_state, done in mini_batch:
                    if(done):
                        target = reward
                    else:
                        target = reward + self.gamma*self.model.predict(next_state)[0][np.argmax(self.target_model.predict(next_state)[0])]
                  
                    q_values = self.model.predict(next_state)
                    q_values[0][action] = target
                  
                    X_target.append(state[0])
                    y_target.append(q_values[0])
                  
            q_values = self.model.predict(state)
            q_values[0][action] = target
                  
            X_train.append(state[0])
            y_train.append(q_values[0])
        
        else:
            print("Bullshit")
        
        loss = self.model.fit(
            np.array(X_train), np.array(y_train), epochs=1, verbose=0
        ).history(["loss"])[0]
        
        if(self.epsilon > self.epsilon_min):
                  self.epsilon *= self.epsilon_decay
        
        return loss
        
                  
    def save(self, episode):
        self.models.save("models/{}_{}".format(self.model_name, episode))
    
    def load(self):
        return load_model("models/" + self.model_name, custom_objects = self.custom_objects) # ??
                  
        

## ***Some Important Utilities***

In [23]:
# Formats Position
format_position = lambda price: ('-$' if price < 0 else '+$') + '{0:.2f}'.format(abs(price))


# Formats Currency
format_currency = lambda price: '${0:.2f}'.format(abs(price))


def show_train_result(result, val_position, initial_offset):
    if(val_position == initial_offset or val_position == 0.0):
        logging.info('Episode {}/{} - Train Position: {} Val Position: USELESS Train Loss: {:.4f}'
                     .format(result[0], result[1], format_position(result[2]), result[3]))
    else:
        logging.info('Episode {}/{} - Train Position: {}  Val Position: {}  Train Loss: {:.4f})'
                     .format(result[0], result[1], format_position(result[2]), format_position(val_position), result[3],))
        

def show_eval_result(model_name, profit, initial_offset):
    """ Displays eval results
    """
    if profit == initial_offset or profit == 0.0:
        logging.info('{}: USELESS\n'.format(model_name))
    else:
        logging.info('{}: {}\n'.format(model_name, format_position(profit)))
        
def get_stock_data(stock_file):
    """Reads stock data from csv file
    """
    df = pd.read_csv(stock_file)
    return list(df['Adj Close'])

def switch_k_backend_device():
    """ Switches `keras` backend from GPU to CPU if required.
    Faster computation on CPU (if using tensorflow-gpu).
    """
    if K.backend() == "tensorflow":
        logging.debug("switching to TensorFlow for CPU")
        os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## ***Some Mathematical Functions***

In [31]:
def sigmoid(x):
    return 1/(1 + math.exp(-x))

# Returns an n-day state representation ending at time t
def get_state(data, t, n_days):
    d = t - n_days + 1
    if (d>=0):
        block = data[d:t+1]
    else:
        block = -d * [data[0]] + data[0:t+1]

    res = []
    for i in range(n_days - 1):
        res.append(sigmoid(block[i+1] - block[i]))
    
    return np.array(res)
    

## ***Model Training Functions***

In [87]:
def train_model(agent, episode, data, ep_count = 100, batch_size = 32, window_size = 10):
    total_profit = 0
    data_length = len(data) - 1
    
    agent.inventory = []
    avg_loss = []
    state = get_state(data, 0, window_size + 1)
    
    
    for t in tqdm(range(data_length), total = data_length, leave = True, desc = 'Episode{}/{}'.format(episode, ep_count)):
        reward = 0
        next_state = get_state(data, t+1, window_size+1)
        
        
        #select an action
        action = agent.act(state)
        
        if(action == 1):
            agent.inventory.append(data[t])
        elif(action == 2) and len(agent.inventory) > 0:
            bought_price = agent.inventory.pop(0)
            delta = data[t] - bought_price
            reward = delta 
            total_profit += delta
            
        else:
            pass
    
    done = (t == data_length-1)
    agent.remember(state, action, reward, next_state, done)
    
    if(len(agent.memory) > batch_size):
        loss = agent.train_experience_replay(batch_size)
        avg_loss.append(loss)
    
    state = next_state
    
    if episode % 10 == 0:
        agent.save(episode)

    return (episode, ep_count, total_profit, np.mean(np.array(avg_loss)))
    
def evaluate_model(agent, data, window_size, debug):
    total_profit = 0
    data_length = len(data) - 1
    
    history = []
    agent.inventory = []
    
    state = get_state(data, 0, window_size + 1)
    
    for t in range(data_length):
        reward = 0
        next_state = get_state(data, t+1, window_size+1)
        # selec an action
        action = agent.act(state, is_eval = True)
        
        if(action == 1):
            agent.inventory.append(data[t])
            history.append((data[t], "BUY"))
            if(debug):
                logging.debug("Buy at: {}".format(format_currency(data[t])))
        
        if(action == 2):
            bought_price = agent.inventory.pop(0)
            delta = data[t] - bought_price
            reward = delta #max(delta, 0)
            total_profit += delta
            
            
            history.append((data[t], "SELL"))
            if debug:
                logging.debug("Sell at: {} | Position: {}".format(
                    format_currency(data[t]), format_position(data[t] - bought_price)))
        
        else:
            history.append((data[t], "HOLD"))
    
        done = (t == data_length - 1)
        agent.memory.append((state, action, reward, next_state, done))
        
        state = next_state
        
        if(done):
            return total_profit, history
            
    

In [89]:
def main_train(train_stock, val_stock, window_size, batch_size, ep_count,
         strategy="t-dqn", model_name="model_debug", pretrained=False,
         debug=False):
    """ Trains the stock trading bot using Deep Q-Learning.
    Please see https://arxiv.org/abs/1312.5602 for more details.
    Args: [python train.py --help]
    """
    agent = Agent(window_size, strategy=strategy, pretrained=pretrained, model_name=model_name)
    
    train_data = get_stock_data(train_stock)
    val_data = get_stock_data(val_stock)

    initial_offset = val_data[1] - val_data[0]

    for episode in range(1, ep_count + 1):
        train_result = train_model(agent, episode, train_data, ep_count=ep_count,
                                   batch_size=batch_size, window_size=window_size)
        val_result, _ = evaluate_model(agent, val_data, window_size, debug)
        show_train_result(train_result, val_result, initial_offset)


if __name__ == "__main__":

    train_stock = "data/GOOG.csv"
    val_stock = "data/GOOG_2018.csv"
    strategy = "t-dqn"
    window_size = 10
    batch_size = 32
    ep_count = 100
    model_name = "my_model"
    pretrained = False
    debug = False
#     model_name = args["--model-name"]
#     pretrained = args["--pretrained"]
#     debug = args["--debug"]

    coloredlogs.install(level="DEBUG")
    switch_k_backend_device()

    try:
        main_train(train_stock, val_stock, window_size, batch_size,
             ep_count, strategy=strategy, model_name=model_name, 
             pretrained=pretrained, debug=debug)
    except KeyboardInterrupt:
        print("Aborted!")

2023-01-14 12:17:25 DESKTOP-IPK9GNB root[236] DEBUG switching to TensorFlow for CPU


Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_105 (Dense)           (None, 128)               1408      
                                                                 
 dense_106 (Dense)           (None, 256)               33024     
                                                                 
 dense_107 (Dense)           (None, 256)               65792     
                                                                 
 dense_108 (Dense)           (None, 128)               32896     
                                                                 
 dense_109 (Dense)           (None, 3)                 387       
                                                                 
Total params: 133,507
Trainable params: 133,507
Non-trainable params: 0
_________________________________________________________________
None


Episode1/100:   0%|          | 0/1760 [00:00<?, ?it/s]

[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0

[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0

...

[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0

Episode1/100: 100%|██████████| 1760/1760 [00:00<00:00, 2930.00it/s]
/home/armat/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/armat/.local/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0

2023-01-14 12:17:26 DESKTOP-IPK9GNB tensorflow[236] WARNING Model was constructed with shape (None, 10) for input KerasTensor(type_spec=TensorSpec(shape=(None, 10), dtype=tf.float32, name='dense_105_input'), name='dense_105_input', description="created by layer 'dense_105_input'"), but it was called on an input with incompatible shape (None,).


ValueError: in user code:

    File "/home/armat/.local/lib/python3.8/site-packages/keras/engine/training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "/home/armat/.local/lib/python3.8/site-packages/keras/engine/training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/armat/.local/lib/python3.8/site-packages/keras/engine/training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "/home/armat/.local/lib/python3.8/site-packages/keras/engine/training.py", line 2079, in predict_step
        return self(x, training=False)
    File "/home/armat/.local/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/armat/.local/lib/python3.8/site-packages/keras/engine/input_spec.py", line 250, in assert_input_compatibility
        raise ValueErr